# ***Домашнє завдання до модуля «Вступ до NLP»***

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

In [ ]:
file_path = '/content/drive/MyDrive/DeepLearningforComputervisionandNLP/spam.csv'
df = pd.read_csv(file_path, encoding='latin1')
df

In [ ]:
df = df[['v1', 'v2']]
df.columns = ['label', 'text']

sns.countplot(x='label', data=df)

In [ ]:
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:
negations = {
    'aren',
    "aren't",
    'couldn',
    "couldn't",
    'didn',
    "didn't",
    'doesn',
    "doesn't",
    'don',
    "don't",
    'hadn',
    "hadn't",
    'hasn',
    "hasn't",
    'haven',
    "haven't",
    'isn',
    "isn't",
    'mightn',
    "mightn't",
    'mustn',
    "mustn't",
    'needn',
    "needn't",
    'no',
    'nor',
    'not',
    'shan',
    "shan't",
    'shouldn',
    "shouldn't",
    'wasn',
    "wasn't",
    'weren',
    "weren't",
    'won',
    "won't",
    'wouldn',
    "wouldn't"
}

In [ ]:
# import nltk
# nltk.download('stopwords')

stop_words = set(stopwords.words('english')).union({'also', 'would', 'much', 'many'}).difference(negations)

stop_words

In [ ]:
def normalize_text(raw_review):

    # Remove html tags
    text = re.sub(r"<[^>]*>", " ", raw_review) # match <> and everything in between. [^>] - match everything except >

    # Remove emails
    text = re.sub(r"\S*@\S*[\s]+", " ", text) # match non-whitespace characters, @ and a whitespaces in the end

    # remove links
    text = re.sub(r"https?:\/\/.*?[\s]+", " ", text) # match http, s - zero or once, //,
                                                    # any char 0-unlimited, whitespaces in the end

     # Convert to lower case, split into individual words
    text = text.lower().split()

    # Replace contractions with their full versions
    text = [contractions.get(word) if word in contractions else word
            for word in text]

    # Re-splitting for the correct stop-words extraction
    text = " ".join(text).split()

    # Remove stop words
    text = [word for word in text if not word in stop_words]

    text = " ".join(text)

    # Remove non-letters
    text = re.sub(r"[^a-zA-Z' ]", "", text) # match everything except letters and '

    # Remove excesive whitespaces
    text = re.sub(r"[\s]+", " ", text)

    # Join the words back into one string separated by space, and return the result.
    return(text)

df = df.assign(clean_text=df['text'].apply(normalize_text))
df

In [ ]:
X = df['clean_text']
y = df['label'].map({'ham': 0, 'spam': 1})

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
vectorizer_bow = CountVectorizer(max_features=3000, ngram_range=(1,2))
X_train_bow = vectorizer_bow.fit_transform(X_train)
X_val_bow = vectorizer_bow.transform(X_val)

In [ ]:
vectorizer_tfidf = TfidfVectorizer(max_features=3000, ngram_range=(1,2))
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_val_tfidf = vectorizer_tfidf.transform(X_val)

In [ ]:
# Встановити якщо відсутні при запуску всього відразу треба час окремо на встановлення
!wget -P /content/saved_models https://nlp.stanford.edu/data/glove.6B.zip
!unzip /content/saved_models/glove.6B.zip -d /content/saved_models/

In [ ]:
EMBEDDING_PATH = '/content/saved_models/glove.6B.50d.txt'

embedding_index = {}
with open(EMBEDDING_PATH, encoding="utf-8") as f:
    for line in f:
        values = line.rstrip().split(" ")
        word = values[0]
        vector = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = vector

In [ ]:
EMBEDDING_DIM = 50

def get_embedding_vector(text):
    vectors = [
        embedding_index[word]
        for word in text.split()
        if word in embedding_index
    ]
    if not vectors:
        return np.zeros(EMBEDDING_DIM)
    return np.mean(vectors, axis=0)

In [ ]:
X_train_embed = np.vstack(X_train.apply(get_embedding_vector))
X_val_embed = np.vstack(X_val.apply(get_embedding_vector))

In [ ]:
# RandomForestClassifier - BoW
model_bow_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_bow_rf.fit(X_train_bow, y_train)
y_pred_bow_rf = model_bow_rf.predict(X_val_bow)

# RandomForestClassifier - TF-IDF
model_tfidf_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_tfidf_rf.fit(X_train_tfidf, y_train)
y_pred_tfidf_rf = model_tfidf_rf.predict(X_val_tfidf)

# RandomForestClassifier - GloVe Embeddings
model_embed_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_embed_rf.fit(X_train_embed, y_train)
y_pred_embed_rf = model_embed_rf.predict(X_val_embed)

# Logistic Regression - BoW
model_bow_lr = LogisticRegression(max_iter=1000)
model_bow_lr.fit(X_train_bow, y_train)
y_pred_bow_lr = model_bow_lr.predict(X_val_bow)

# Logistic Regression - TF-IDF
model_tfidf_lr = LogisticRegression(max_iter=1000)
model_tfidf_lr.fit(X_train_tfidf, y_train)
y_pred_tfidf_lr = model_tfidf_lr.predict(X_val_tfidf)

# Logistic Regression - GloVe Embeddings
model_embed_lr = LogisticRegression(max_iter=1000)
model_embed_lr.fit(X_train_embed, y_train)
y_pred_embed_lr = model_embed_lr.predict(X_val_embed)

In [ ]:
# Функція оцінки моделей
def evaluate_model(y_true, y_pred, model_name):
    acc = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    print(f"{model_name} — Accuracy: {acc:.4f}, AUC: {auc:.4f}")
    print(classification_report(y_true, y_pred))

evaluate_model(y_val, y_pred_bow_rf, "RF BoW")
evaluate_model(y_val, y_pred_tfidf_rf, "RF TF-IDF")
evaluate_model(y_val, y_pred_embed_rf, "RF GloVe Embeddings")

evaluate_model(y_val, y_pred_bow_lr, "LR BoW")
evaluate_model(y_val, y_pred_tfidf_lr, "LR TF-IDF")
evaluate_model(y_val, y_pred_embed_lr, "LR GloVe Embeddings")

In [ ]:
# Таблиця результатів моделей, що використовують BoW та TF-IDF, та попередньо навчені ембединги
results_table = pd.DataFrame({
    "Модель": ["RF BoW", "RF TF-IDF", "RF GloVe Embeddings", "LR BoW", "LR TF-IDF", "LR GloVe Embeddings"],
    "Accuracy": [
        accuracy_score(y_val, y_pred_bow_rf),
        accuracy_score(y_val, y_pred_tfidf_rf),
        accuracy_score(y_val, y_pred_embed_rf),
        accuracy_score(y_val, y_pred_bow_lr),
        accuracy_score(y_val, y_pred_tfidf_lr),
        accuracy_score(y_val, y_pred_embed_lr)
    ],
    "AUC": [
        roc_auc_score(y_val, y_pred_bow_rf),
        roc_auc_score(y_val, y_pred_tfidf_rf),
        roc_auc_score(y_val, y_pred_embed_rf),
        roc_auc_score(y_val, y_pred_bow_lr),
        roc_auc_score(y_val, y_pred_tfidf_lr),
        roc_auc_score(y_val, y_pred_embed_lr)
    ]
})
results_table

# ***ВИСНОВКИ***

# **Оцінка моделей**

Для валідаційного набору даних були побудовані прогнози з використанням моделей RandomForestClassifier та LogisticRegression на трьох типах ознак: BoW, TF-IDF та GloVe embeddings.


**Метрики ефективності**
*BoW та TF-IDF показали найвищі результати:*

* Accuracy: ~97-98%
* AUC: ~0.90-0.92



*GloVe embeddings дали нижчі показники:*

* Accuracy: ~92-95%
* AUC: ~0.79-0.86

# ***Обґрунтування вибору метрик***

* Accuracy дозволяє оцінити загальну точність класифікації.

* AUC (Area Under the Curve) краще відображає якість моделі при різних порогах класифікації, особливо важливо задачі спаму, де баланс класів може бути нерівномірним.

# ***Порівняння результатів***

* Logistic Regression на BoW показала найкращий результат (Accuracy ≈ 98%, AUC ≈ 0.92).

* RandomForest на TF-IDF також показав високу якість (Accuracy ≈ 97.6%, AUC ≈ 0.91).

* Моделі на GloVe embeddings поступаються, оскільки усереднені вектори втрачають важливі слова‑індикатори спаму.

# ***Аналіз та інтерпретація результатів***

**Найкраща модель**

* Logistic Regression з BoW - найефективніша для класифікації спаму на цьому наборі даних.


**Переваги та недоліки підходів**

*BoW*
  * Простий для реалізації, добре працює для задач, де важливі конкретні слова.
  * Але не  враховує семантику та порядок слів.

*TF-IDF*
  * Враховує важливість слів контексту, зменшує вплив частих, але малозначущих слів.
  * Як BoW, не відображає семантичні зв’язки.

*GloVe embeddings*
  * Враховують семантику та схожість слів, корисні для більш складних задач NLP.
  * Усереднення векторів призводить до втрати ключових ознак, що критично для задачі спаму.

Для задачі класифікації спаму найкраще працюють класичні методи BoW та TF-IDF поєднанні з Logistic Regression. Попередньо навчені embeddings менш ефективні, оскільки семантична інформація не настільки важлива, як наявність конкретних ключових слів.